In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_digits

In [3]:
dataset = pd.read_csv('/Users/pawankumarkc/Documents/vscode_workspace/MLAlgo/datasets/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset['Exited'].value_counts()

#Imbalanced data

0    7963
1    2037
Name: Exited, dtype: int64

In [5]:
dataset = dataset.iloc[:,3:]
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dataset.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [8]:
dataset = pd.get_dummies(dataset, columns=['Geography','Gender'], drop_first=True)

In [9]:
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [10]:
## Split the data into dep and ind variable

x = dataset.drop(['Exited'], axis=1) 
y = dataset[['Exited']]

In [11]:
# Balance the data

from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)
print(y.value_counts())
print(y_smote.value_counts())

Exited
0         7963
1         2037
dtype: int64
Exited
0         7963
1         7963
dtype: int64


In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=0.25, random_state=101)

In [13]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
#Hyperparam tuning

#1. How many no. of hidden layer we should have?
#2. How many no. of neurons we should have in each hidden layer?
#3. Which activation to use in each hidden layer?
#4. What is the best learning rate?

from kerastuner.tuners import RandomSearch


In [14]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

In [ ]:
# Multi layer preceptron

model = Sequential()
model.add(Dense(32, activation='relu',input_dim=11))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.summary()

In [28]:
def build_model(hp):
    model = Sequential()
    #Assign hyperparam tuning - how many hidden layer??
    counter=0
    for i in range(hp.Int('no_hidden_layer', min_value=1, max_value=10)):
        if counter==0:
            model.add(Dense(hp.Int('no_nuerons'+ str(i), min_value=8, max_value=128, step=8), 
                                   activation=hp.Choice('activation_func'+str(i), values=['relu','leaky_relu']), input_dim=11))
            model.add(Dropout(hp.Choice("Dropout"+str(i), values=[0.1,0.2,0.3,0.4,0.5])))
        else:
            model.add(Dense(hp.Int('no_nuerons'+ str(i), min_value=8, max_value=128, step=8), 
                        activation=hp.Choice('activation_func'+str(i), values=['relu','leaky_relu'])))
            model.add(Dropout(hp.Choice("Dropout"+str(i), values=[0.1,0.2,0.3,0.4,0.5])))
        counter+=1

    model.add(Dense(1, activation='sigmoid')) # output layer, binary class= sigmoid
    model.compile(optimizer=hp.Choice("Optimizer", values=['adam', 'rmsprop','sgd']),
                   loss='binary_crossentropy', metrics=['accuracy'])
    return model
    

In [29]:
from kerastuner.tuners import RandomSearch
import kerastuner as kt

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_own_dir', project_name='HyperParam'
)

In [30]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.5657960772514343

Best val_accuracy So Far: 0.5673028826713562
Total elapsed time: 00h 00m 07s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'no_hidden_layer': 1,
 'no_nuerons0': 112,
 'activation_func0': 'leaky_relu',
 'Dropout0': 0.2,
 'Optimizer': 'adam',
 'no_nuerons1': 128,
 'activation_func1': 'relu',
 'Dropout1': 0.1,
 'no_nuerons2': 104,
 'activation_func2': 'relu',
 'Dropout2': 0.5,
 'no_nuerons3': 120,
 'activation_func3': 'leaky_relu',
 'Dropout3': 0.4,
 'no_nuerons4': 40,
 'activation_func4': 'leaky_relu',
 'Dropout4': 0.3,
 'no_nuerons5': 32,
 'activation_func5': 'leaky_relu',
 'Dropout5': 0.2}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

In [33]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=5, validation_data=(x_test, y_test))

Epoch 6/100
374/374 [==============================] - 0s 559us/step - loss: 138.3109 - accuracy: 0.5110 - val_loss: 25.3471 - val_accuracy: 0.4992
Epoch 7/100
374/374 [==============================] - 0s 422us/step - loss: 49.9668 - accuracy: 0.5201 - val_loss: 17.5625 - val_accuracy: 0.5266
Epoch 8/100
374/374 [==============================] - 0s 421us/step - loss: 24.6668 - accuracy: 0.5233 - val_loss: 11.3523 - val_accuracy: 0.5678
Epoch 9/100
374/374 [==============================] - 0s 424us/step - loss: 19.0744 - accuracy: 0.5267 - val_loss: 22.2065 - val_accuracy: 0.5306
Epoch 10/100
374/374 [==============================] - 0s 429us/step - loss: 15.6286 - accuracy: 0.5280 - val_loss: 19.6709 - val_accuracy: 0.5093
Epoch 11/100
374/374 [==============================] - 0s 432us/step - loss: 13.0507 - accuracy: 0.5308 - val_loss: 2.8307 - val_accuracy: 0.5972
Epoch 12/100
374/374 [==============================] - 0s 425us/step - loss: 12.2712 - accuracy: 0.5234 - val_loss:

## pytorch DNN

In [35]:
import torch
from torch import nn    #neural network
from torch.nn import functional as f    #functional api
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

#Brest cancer datset - classification
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

from tqdm import tqdm #Progress package

In [36]:
#loading dataset

data = load_breast_cancer()
data

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [37]:
x = data['data']
y = data['target']

print(f"Size x = {x.shape}")
print(f"Size y =  {y.shape}")


Size x = (569, 30)
Size y =  (569,)


In [40]:
pd.DataFrame(x).head

<bound method NDFrame.head of         0      1       2       3        4        5        6        7       8   \
0    17.99  10.38  122.80  1001.0  0.11840  0.27760  0.30010  0.14710  0.2419   
1    20.57  17.77  132.90  1326.0  0.08474  0.07864  0.08690  0.07017  0.1812   
2    19.69  21.25  130.00  1203.0  0.10960  0.15990  0.19740  0.12790  0.2069   
3    11.42  20.38   77.58   386.1  0.14250  0.28390  0.24140  0.10520  0.2597   
4    20.29  14.34  135.10  1297.0  0.10030  0.13280  0.19800  0.10430  0.1809   
..     ...    ...     ...     ...      ...      ...      ...      ...     ...   
564  21.56  22.39  142.00  1479.0  0.11100  0.11590  0.24390  0.13890  0.1726   
565  20.13  28.25  131.20  1261.0  0.09780  0.10340  0.14400  0.09791  0.1752   
566  16.60  28.08  108.30   858.1  0.08455  0.10230  0.09251  0.05302  0.1590   
567  20.60  29.33  140.10  1265.0  0.11780  0.27700  0.35140  0.15200  0.2397   
568   7.76  24.54   47.92   181.0  0.05263  0.04362  0.00000  0.00000  0.1587  

In [42]:
pd.DataFrame(y).value_counts()  #Data is balanced

1    357
0    212
dtype: int64

In [61]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=42, stratify=y)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((426, 30), (426,), (143, 30), (143,))

In [62]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [63]:
class Hyperparameters():
    def __init__(self):
        self.number_of_epochs = 50
        self.batch_size=8
        self.learning_rate=0.01
        self.shuffle = True

In [64]:
#Train data

class TrainData(Dataset):
    def __init__(self, x_train, y_train):
        self.x_train = torch.tensor(x_train, dtype=torch.float32)
        self.y_train = torch.tensor(y_train, dtype=torch.float32)

    def __getitem__(self, index):
        return self.x_train[index], self.y_train[index]
    
    def __len__(self):
        return len(self.x_train)
    
train_data = TrainData(x_train, y_train)



In [65]:
#Test data

class TestData(Dataset):
    def __init__(self, x_test):
        self.x_train = torch.tensor(x_test, dtype=torch.float32)

    def __getitem__(self, index):
        return self.x_test[index]
    
    def __len__(self):
        return len(self.x_test)
    
test_data = TestData(x_test)

In [66]:
#Declaring the train and test dataset loaders

hyperparameters = Hyperparameters()
train_loader = DataLoader(dataset= train_data, batch_size=hyperparameters.batch_size, 
                          shuffle=hyperparameters.shuffle)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [67]:
#Define the model

class BrestCancerClassification(nn.Module):
    def __init__(self, input_shape):
        super(BrestCancerClassification, self).__init__()

        self.fc1 = nn.Linear(input_shape, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))

        return x

In [68]:
#Checking the GPU
!nvidia -smi

zsh:1: command not found: nvidia


In [69]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [70]:
x_train.shape

(426, 30)

In [72]:
#building model
model = BrestCancerClassification(input_shape=x_train.shape[1])
#declaring the Optimizer
optimizer = optim.Adam(model.parameters(), lr = hyperparameters.learning_rate)
#declaring the loss function
criterian = nn.BCELoss()
#moving model to device
model.to(device)

BrestCancerClassification(
  (fc1): Linear(in_features=30, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
)

In [73]:
#This function is responsible for calculating the accuracy for a given batch

def binary_acc(y_pred, y):
    results = torch.round(y_pred)
    correct_results_sum = (results == y).sum().float()
    acc = correct_results_sum / y.shape[0]
    acc = torch.round(acc*100)


In [74]:
#Training the model 
losses = []
accuracies = []
model.train()

#putting the model in train mode
for i in range(1, hyperparameters.number_of_epochs+1):
    epoch_loss = 0
    epoch_accuracy = 0
    for data in train_loader:
        x_train, y_train = data
        x_train = x_train.to(device)    #Moving the input features to device
        y_train = y_train.to(device)

        optimizer.zero_grad()
        #Zeroing the gradinets so that they don't accumulate the next batch

        y_pred = model(x_train)
        loss = criterian(y_pred, y_train.reshape(-1,1))

        acc = binary_acc(y_pred, y_train.reshape(-1,1))

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_accuracy += acc.item()

    if i % 5 == 0:
        print(f'Epoch {i+0.3}: | Loss: {epoch_loss / len(train_loader):.5f} | Acc: {epoch_accuracy/len(train_loader):.3f }')
        losses.append(epoch_loss/len(train_loader))
        accuracies.append(epoch_loss/len(train_loader))
        


AttributeError: 'NoneType' object has no attribute 'item'

In [ ]:
#plotting the losses

